# Python语言与经济大数据分析

# 第6_1讲 数据处理:中国知网数据处理

#郭峰（上海财经大学投资系副教授）
#Email：guo.feng@mail.sufe.edu.cn
#2019-02-26
#本讲主要以中国知网数据为例，讲解数据整理的基本操作，主要是dataframe，涉及数据清洗、增加、删除、合并等等。
#数据源：CSSCI期刊，含拓展版，2001-2017年，200余万原始爬虫记录
#6.1.1 通用清理
#6.1.2 科研人员迁移识别
#6.1.3 关系稿识别

# 6.1.1 通用清理

In [3]:
# 首先要对相关数据进行清洗，删除各种异常值、重复值等
#使用pandas库，学习资料：http://m.sohu.com/a/158827610_693397
#pandas里定义的数据类型： （1.）object字符值（2.）int整型（3.）float浮点型（4.）datatime时间值（5.）bool布尔值
# pandas数据清洗参考资料：https://www.cnblogs.com/stream886/p/6021743.html
#https://blog.csdn.net/claroja/article/details/65661826
import pandas as pd
import os
import numpy as np
import pypinyin
import datetime
starttime = datetime.datetime.now()

#实际工作
'''
path="D:/python/cnki/"
f1 = open(path+"cssci_org1.csv",encoding='utf8')
papers1= pd.read_csv(f1,header=0,sep=',')
#数据导入：https://www.cnblogs.com/OliverQin/p/8966321.html

##pandas数据拼接：https://blog.csdn.net/stevenkwong/article/details/52528616
f2 = open(path+"cssci_org2.csv",encoding='utf8')
papers2= pd.read_csv(f2,header=0,sep=',')

f3 = open(path+"cssci_org3.csv",encoding='utf8')
papers3= pd.read_csv(f3,header=0,sep=',')   #不同电脑提取数据时方式不同，这里分隔符不同，\t代表制表符。

f4 = open(path+"cssci_org4.csv",encoding='utf8')
papers4= pd.read_csv(f4,header=0,sep=',')


f5 = open(path+"cssci_org5.csv",encoding='utf8')
papers5= pd.read_csv(f5,header=0,sep=',')

f6 = open(path+"cssci_org6.csv",encoding='utf8')  #cssci拓展版
papers6= pd.read_csv(f6,header=0,sep=',')

f7 = open(path+"cssci_org7.csv",encoding='utf8') #cssci拓展版
papers7= pd.read_csv(f7,header=0,sep=',')

f8 = open(path+"cssci_org8.csv",encoding='utf8') #cssci拓展版
papers8= pd.read_csv(f8,header=0,sep=',')

f9 = open(path+"cssci_org9.csv",encoding='utf8') #cssci拓展版
papers9= pd.read_csv(f9,header=0,sep=',')

papers= papers1.append(papers2)
papers= papers.append(papers3)
papers= papers.append(papers4)
papers= papers.append(papers5)
papers= papers.append(papers6)
papers= papers.append(papers7)
papers= papers.append(papers8)
papers= papers.append(papers9)
#papers=papers1 #如果只研究经管类，就只使用第一个数据
papers.drop('id',inplace=True) 

'''
#教学演示
path='D:/python/郭峰Python讲义/数据与结果/cssci/'
f = open(path+"cssci_org_test.csv",encoding='utf8')
papers= pd.read_csv(f,header=0,sep=',')

#作为示例，输出CSV文件的前5行和最后5行，这是pandas默认的输出5行，可以根据需要自己设定输出几行的值
print(papers.head())
print(papers.tail())
print(papers.shape)

print("原始爬虫数量:"+str(len(papers))) #查看行

#id没有意义，删除，删除多个变量：papers.drop(['id','title'],inplace=True) 
#凡是会对原数组作出修改并返回一个新数组的，往往都有一个 inplace可选参数。
#如果手动设定为True（默认为False），那么原数组直接就被替换。
#而采用inplace=False之后，原数组名对应的内存值并不改变，需要将新的结果赋给一个新的数组papers2=papers.drop('id') 

#删除爬虫时的重复值，以标题、期刊名和作者同时重复为准
#删除数据参考：https://blog.csdn.net/qq_28811329/article/details/79962511
papers.drop_duplicates(subset=['title','author','mag_name'],keep='first',inplace=True) 
print("删除重复后数量:"+str(len(papers))) 

#删除非论文文章：删除标题中还有以下字段的数据
#删除数据的几种情况：https://www.cnblogs.com/cocowool/p/8421997.html
papers=papers[~papers['title'].str.contains('卷首语')]
papers=papers[~papers['title'].str.contains('寄语')]
papers=papers[~papers['title'].str.contains('发刊词')]
papers=papers[~papers['title'].str.contains('序言')]
papers=papers[~papers['title'].str.contains('简讯')]
papers=papers[~papers['title'].str.contains('会讯')]
papers=papers[~papers['title'].str.contains('会议简述')]
papers=papers[~papers['title'].str.contains('会议综述')]
papers=papers[~papers['title'].str.contains('会议纪实')]
papers=papers[~papers['title'].str.contains('研讨会')]
papers=papers[~papers['title'].str.contains('名单')]
papers=papers[~papers['title'].str.contains('纪要')]
papers=papers[~papers['title'].str.contains('诞辰')]
papers=papers[~papers['title'].str.contains('悼词')]
papers=papers[~papers['title'].str.contains('评奖')]
papers=papers[~papers['title'].str.contains('目录')]
papers=papers[~papers['title'].str.contains('启事')]
papers=papers[~papers['title'].str.contains('广告')]
papers=papers[~papers['title'].str.contains('公告')]

#删除标题异常样本
papers["tlength"]=papers['title'].str.len()  #标题长度，不能是papers["tlength"]=len(papers['title'])
papers=papers[papers['tlength'] <100]  #删除标题长度大于100的异常值
papers=papers[papers['tlength'] >2]  #删除标题长度小于2的异常值
print("标题剔除后数量:"+str(len(papers))) 

#删除长度异常样本
papers=papers[(papers['page_num'] >0) & (papers['page_num'] <100)]
print("页码异常剔除后数量:"+str(len(papers))) 
#其他padas运算：https://www.jianshu.com/p/805f20ac6e06

#删除作者异常样本
papers.dropna(subset=["author"],inplace=True)  #删除作者为空的样本
papers=papers[~papers['author'].str.contains('编辑部')]   #删除作者信息中包含“编辑部”的样本。
#删除空值的另一种方式，将df中A列所有空值赋值为'null'，然后再删除
#papers['author']=papers['author'].fillna('null') 
#papers=papers[~papers['author'].isin(['null'])]
papers["aunum"] =papers['author'].str.count(";")+1
papers=papers[papers['aunum'] <20]    #删除作者数量大于等于20的样本
print("作者异常剔除后数量:"+str(len(papers))) #查看行*列数

#第一作者识别，这个涉及到pandas分列
papers['author_first']=papers['author'].str.split(';',expand=True)[0]

#第一作者作者首写字母,一个个循环，非常耗时，excel表格可以计算（百度：getpy）
#papers['index']=range(papers.shape[0])  #之前的index序号不连贯了
#papers.set_index('index')
#papers["auorder"]=None
#for i in range(0,len(papers)):
#    if len(papers['author_first'])>0:
#        papers['auorder'][i]=pypinyin.pinyin(papers['author_first'][i], style=pypinyin.NORMAL)[0][0][0]
#    else:
#        pass

#关键词和资助信息整理
papers["keyword"] =papers['keyword'].str.replace("关键词：","")
papers["keyword"] =papers['keyword'].str.replace("\n","")
papers["kwnum"] =papers['keyword'].str.count(";")
papers['ablength']=papers['abstract'].str.len()
papers["fund"] =papers['fund'].str.replace("基金：","")
papers["fund01"] =(papers['fund'].str.len()>0)
papers["fundn"]=((papers['fund'].str.contains('国家社科'))|(papers['fund'].str.contains('国家自科'))|(papers['fund'].str.contains('国家社会科学'))|(papers['fund'].str.contains('国家自然科学')))

#期刊名称整理
papers['mag_name']=papers['mag_name'].str.split('-',expand=True)[0]
papers['mag_name']=papers['mag_name'].str.split('.',expand=True)[0]
papers["mag_name"] =papers['mag_name'].str.replace("(","")
papers["mag_name"] =papers['mag_name'].str.replace(")","")
papers["mag_name"] =papers['mag_name'].str.replace("\n","")  #部分期刊名称存在换行的异常
papers=papers[~papers['mag_name'].str.contains('China & World Economy')]
papers=papers[~papers['mag_name'].str.contains('China Legal Science')]
papers=papers[~papers['mag_name'].str.contains('湘潭大学社会科学学报研究生论丛')]
papers=papers[~papers['mag_name'].str.contains('中央民族大学学报自然科学版')]
papers=papers[~papers['mag_name'].str.contains('商情财经研究')]
papers=papers[~papers['mag_name'].str.contains('科技广场管理科学')]
papers=papers[~papers['mag_name'].str.contains('云南经济管理干部学院学报')]
papers=papers[~papers['mag_name'].str.contains('山东农业农村经济')]
papers=papers[~papers['mag_name'].str.contains('中国电视纪录')]
papers=papers[~papers['mag_name'].str.contains('公民与法法学')]
papers=papers[~papers['mag_name'].str.contains('China International Studies')]
papers=papers[~papers['mag_name'].str.contains('新课程研究教育研究与实验')]
papers=papers[~papers['mag_name'].str.contains('北京联合大学学报自然科学版')]
papers=papers[~papers['mag_name'].str.contains('北京联合大学学报教育教学研究专辑')]
papers=papers[~papers['mag_name'].str.contains('山西财经大学学报高等教育版')]
#以下为cssci拓展版
papers=papers[~papers['mag_name'].str.contains('Chinese Journal of Library and Information Science')]
papers=papers[~papers['mag_name'].str.contains('上海包装')]
papers=papers[~papers['mag_name'].str.contains('药物评价研究')]
papers=papers[~papers['mag_name'].str.contains('智能系统学报')]
papers=papers[~papers['mag_name'].str.contains('中国战略新兴产业')]
papers=papers[~papers['mag_name'].str.contains('吉林大学学报工学版')]
papers=papers[~papers['mag_name'].str.contains('商丘职业技术学院学报')]
papers=papers[~papers['mag_name'].str.contains('食品科学')]
papers=papers[~papers['mag_name'].str.contains('玉米科学')]
papers=papers[~papers['mag_name'].str.contains('电脑知识与技术学术交流')]
papers=papers[~papers['mag_name'].str.contains('技术经济与管理研究')]

#不规范期刊名称替换
papers["mag_name"] =papers['mag_name'].str.replace("广东金融学院学报","金融经济学研究")
papers["mag_name"] =papers['mag_name'].str.replace("华南金融研究","金融经济学研究")
papers["mag_name"] =papers['mag_name'].str.replace("贵州财经学院学报","贵州财经大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("广东商学院学报","广东财经大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("中南财经大学学报","中南财经政法大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("云南财贸学院学报","云南财经大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("决策借鉴","管理科学")
papers["mag_name"] =papers['mag_name'].str.replace("财经论丛浙江财经学院学报","财经论丛")
papers["mag_name"] =papers['mag_name'].str.replace("现代财经天津财经大学学报","现代财经")
papers["mag_name"] =papers['mag_name'].str.replace("系统工程理论方法应用","系统管理学报")
papers["mag_name"] =papers['mag_name'].str.replace("国际商务对外经济贸易大学学报","国际商务")
papers["mag_name"] =papers['mag_name'].str.replace("中外管理导报","管理评论")
papers["mag_name"] =papers['mag_name'].str.replace("法商研究中南政法学院学报","法商研究")
papers["mag_name"] =papers['mag_name'].str.replace("法律科学西北政法大学学报","法律科学")
papers["mag_name"] =papers['mag_name'].str.replace("法律科学西北政法学院学报","法律科学")
papers["mag_name"] =papers['mag_name'].str.replace("人民论坛·学术前沿","人民论坛")
papers["mag_name"] =papers['mag_name'].str.replace("内蒙古社会科学汉文版","内蒙古社会科学")
papers["mag_name"] =papers['mag_name'].str.replace("南京大学学报哲学·人文科学·社会科学","南京大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("南京大学学报哲学","南京大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("北京师范大学学报人文社会科学版","北京师范大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("山东大学学报人文社会科学版","山东大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("东北师大学报哲学社会科学版","东北师大学报")
papers["mag_name"] =papers['mag_name'].str.replace("西藏大学学报汉文版","西藏大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("南通大学学报哲学社会科学版","南通大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("南通师范学院学报哲学社会科学版","南通大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("北京商学院学报","北京工商大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("杭州商学院学报","浙江工商大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("浙江省政法管理干部学院学报","浙江工商大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("中央民族大学学报","中央民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("中南民族学院学报人文社会科学版","中南民族大学学报人文社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("课程·教材·教法","课程")
papers["mag_name"] =papers['mag_name'].str.replace("历史教学下半月刊","历史教学")
papers["mag_name"] =papers['mag_name'].str.replace("历史教学高校版","历史教学2")
papers["mag_name"] =papers['mag_name'].str.replace("国外外语教学","外语教学理论与实践")
papers["mag_name"] =papers['mag_name'].str.replace("音乐艺术上海音乐学院学报","音乐艺术")
papers["mag_name"] =papers['mag_name'].str.replace("厦门大学学报 哲学社会科学版","厦门大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("南开学报哲学社会科学版","南开学报")
papers["mag_name"] =papers['mag_name'].str.replace("兰州大学学报社会科学版","兰州大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("中南工业大学学报社会科学版","中南大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("安徽大学学报哲学社会科学版","安徽大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("新疆大学学报哲学·人文社会科学版","新疆大学学报哲学人文社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("新疆大学学报哲学社会科学版","新疆大学学报哲学人文社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("江西师范大学学报哲学社会科学版","江西师范大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("华中科技大学学报人文社会科学版","华中科技大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("云南民族学院学报哲学社会科学版","云南民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("广西民族学院学报哲学社会科学版","广西民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("现代传播中国传媒大学学报","现代传播")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报美术与设计版","南京艺术学院学报")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报美术及设计版","南京艺术学院学报")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报美术与设计","南京艺术学院学报")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报美术&设计版","南京艺术学院学报")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报音乐及表演版","南京艺术学院学报2")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报音乐与表演版","南京艺术学院学报2")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报音乐与表演","南京艺术学院学报2")
papers["mag_name"] =papers['mag_name'].str.replace("南京艺术学院学报音乐&表演版","南京艺术学院学报2")
papers["mag_name"] =papers['mag_name'].str.replace("市场与人口分析","人口与发展")
papers["mag_name"] =papers['mag_name'].str.replace("武汉大学学报社会科学版","武汉大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("西南大学学报人文社会科学版","西南大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("西南师范大学学报人文社会科学版","西南大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("苏州大学学报哲学社会科学版","苏州大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("上海交通大学学报社会科学版","上海交通大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("暨南学报人文科学与社会科学版","暨南学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("西南民族大学学报人文社会科学版","西南民族大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("西南民族大学学报人文社科版","西南民族大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("西南民族学院学报哲学社会科学版","西南民族大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("教育与现代化","研究生教育研究")
papers["mag_name"] =papers['mag_name'].str.replace("湘潭大学社会科学学报","湘潭大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("湘潭工学院学报社会科学版","湖南科技大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("中央民族大学学报哲学社会科学版哲学社会科学版","中央民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("课程教材教法教材教法","课程教材教法")
papers["mag_name"] =papers['mag_name'].str.replace("外交评论外交学院学报","外交评论")
papers["mag_name"] =papers['mag_name'].str.replace("外交学院学报","外交评论")
papers["mag_name"] =papers['mag_name'].str.replace("法律科学.西北政法学院学报","法律科学")
papers["mag_name"] =papers['mag_name'].str.replace("修辞学习","当代修辞学")
papers["mag_name"] =papers['mag_name'].str.replace("学科教育","教育学报")
papers["mag_name"] =papers['mag_name'].str.replace("科学技术与辩证法","科学技术哲学研究")
papers["mag_name"] =papers['mag_name'].str.replace("现代图书情报技术","数据分析与知识发现")
papers["mag_name"] =papers['mag_name'].str.replace("华东政法学院学报","华东政法大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("欧洲研究","欧洲")  #理论上应该是欧洲研究是最新的，但颠倒过来有问题
papers["mag_name"] =papers['mag_name'].str.replace("城市规划汇刊","城市规划学刊")
papers["mag_name"] =papers['mag_name'].str.replace("江苏大学学报高教研究版","高校教育管理")
papers["mag_name"] =papers['mag_name'].str.replace("镇江师专学报社会科学版","高校教育管理")
papers["mag_name"] =papers['mag_name'].str.replace("高等师范教育研究","教师教育研究")
papers["mag_name"] =papers['mag_name'].str.replace("北京联合大学学报人文社会科学版","北京联合大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("饰","艺术设计研究")
papers["mag_name"] =papers['mag_name'].str.replace("装艺术设计研究","装饰")  #存在“装饰”和“饰“两种期刊
#以下为CSSCI拓展版
papers["mag_name"] =papers['mag_name'].str.replace("美苑","艺术工作")
papers["mag_name"] =papers['mag_name'].str.replace("中国历史文物","中国国家博物馆馆刊")
papers["mag_name"] =papers['mag_name'].str.replace("南京审计学院学报","南京审计大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("世界贸易组织动态与研究","上海对外经贸大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("陕西经贸学院学报","西安财经学院学报")
papers["mag_name"] =papers['mag_name'].str.replace("重庆工商大学学报西部论坛","西部论坛")
papers["mag_name"] =papers['mag_name'].str.replace("重庆工商大学学报","西部论坛")
papers["mag_name"] =papers['mag_name'].str.replace("重庆商学院学报","西部论坛")
papers["mag_name"] =papers['mag_name'].str.replace("青岛海洋大学学报社会科学版","中国海洋大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("自贡师范高等专科学校学报","上海对外经贸大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("江西教育科研","教育学术月刊")
papers["mag_name"] =papers['mag_name'].str.replace("新乡教育学院学报","管理学刊")
papers["mag_name"] =papers['mag_name'].str.replace("科学对社会的影响","科学与社会")
papers["mag_name"] =papers['mag_name'].str.replace("社会保障研究北京","社会保障研究")
papers["mag_name"] =papers['mag_name'].str.replace("暨南大学华文学院学报","华文教学与研究")
papers["mag_name"] =papers['mag_name'].str.replace("上海科技翻译","上海翻译")
papers["mag_name"] =papers['mag_name'].str.replace("西安外国语学院学报","西安外国语大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("国际关系学院学报","国际安全研究")
papers["mag_name"] =papers['mag_name'].str.replace("中国青年政治学院学报","中国青年社会科学")
papers["mag_name"] =papers['mag_name'].str.replace("税务与经济长春税务学院学报","税务与经济")
papers["mag_name"] =papers['mag_name'].str.replace("西北民族学院学报哲学社会科学版","西北民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("西藏民族学院学报哲学社会科学版","西藏民族大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("黑龙江财专学报","哈尔滨商业大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("国外城市规划","国际城市规划")
papers["mag_name"] =papers['mag_name'].str.replace("中国高校科技与产业化学术版","中国高校科技")
papers["mag_name"] =papers['mag_name'].str.replace("中国高校科技与产业化","中国高校科技")
papers["mag_name"] =papers['mag_name'].str.replace("中国高校技术市场","中国高校科技")
papers["mag_name"] =papers['mag_name'].str.replace("交通高教研究","高教发展与评估")
papers["mag_name"] =papers['mag_name'].str.replace("教育理论与实践学科版","教育理论与实践")
papers["mag_name"] =papers['mag_name'].str.replace("建筑管理现代化","管理现代化")
papers["mag_name"] =papers['mag_name'].str.replace("外国文学动态研究","外国文学动态")
papers["mag_name"] =papers['mag_name'].str.replace("上海立信会计学院学报","会计与经济研究")
papers["mag_name"] =papers['mag_name'].str.replace("立信会计高等专科学校学报","会计与经济研究")
papers["mag_name"] =papers['mag_name'].str.replace("山东经济","经济与管理评论")
papers["mag_name"] =papers['mag_name'].str.replace("阿拉伯世界研究","阿拉伯世界")
papers["mag_name"] =papers['mag_name'].str.replace("戏剧中央戏剧学院学报","戏剧")
papers["mag_name"] =papers['mag_name'].str.replace("北方民族大学学报哲学社会科学版","北方民族大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("戏剧中央戏剧学院学报","戏剧")
papers["mag_name"] =papers['mag_name'].str.replace("法律适用国家法官学院学报","法律适用")
papers["mag_name"] =papers['mag_name'].str.replace("杭州师范学院学报人文社会科学版","杭州师范大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("杭州师范学院学报社会科学版","杭州师范大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("华侨大学学报人文社会科学版","华侨大学学报哲学社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("江苏理工大学学报社会科学版","江苏大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("南京化工大学学报哲学社会科学版","南京工业大学学报社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("山东师大学报人文社会科学版","山东师范大学学报人文社会科学版")
papers["mag_name"] =papers['mag_name'].str.replace("体育文史","体育文化导刊")
papers["mag_name"] =papers['mag_name'].str.replace("西北第二民族学院学报哲学社会科学版","北方民族大学学报")
papers["mag_name"] =papers['mag_name'].str.replace("中学教育","基础教育")
papers["mag_name"] =papers['mag_name'].str.replace("俄罗斯东欧中亚研究","东欧中亚研究")
papers["mag_name"] =papers['mag_name'].str.replace("俄罗斯中亚东欧研究","东欧中亚研究")
papers["mag_name"] =papers['mag_name'].str.replace("机械工业高教研究","大学教育科学")
papers["mag_name"] =papers['mag_name'].str.replace("武汉大学学报社会科学版","武汉大学学报哲学社会科学版")

#期刊名称修改后，再删除一下可能的重复值
papers.drop_duplicates(subset=['title','author','mag_name'],keep='first',inplace=True) 
print("期刊异常剔除后数量:"+str(len(papers))) #查看行*列数

#期刊-主办单位匹配
#pandas merge方法：https://blog.csdn.net/Darkman_EX/article/details/80726166
fmag= open(path+"mag_unit.csv",encoding='utf8')
fmag= pd.read_csv(fmag,header=0,sep=',')
papers=pd.merge(papers,fmag,how='left')

#发表时间整理
papers['year_period']=papers['year_period'].str[0:10]  #部分样本存在“2017-12-15 14：55”这样的year_period
#papers['year_period']=papers['year_period'][0:10]  #如果是这样，则代表只要这个变量的前10行。。。
print(papers['year_period'].tail())
papers['year']=papers['year_period'].str[0:4]
papers['month']=papers['year_period'].str[5:7]
papers['month']=papers['month'].str.extract('(\d+)', expand=False)
#部分日期格式存在问题，正常为2017-06-15，部分样本为2017/6/15,上式将数字“/6"当中的数字提取出来
#pandas字符中提取数字：https://blog.csdn.net/PeersLee/article/details/77948423   
#示例：df[['室', '厅', '厨', '卫']] = df['户型'].str.extract('(\d+)室(\d+)厅(\d+)厨(\d+)卫', expand=False)
papers=papers[papers['year'].str.len()>0]  #部分样本日期存在错误
#年和月数据都转换成整数
papers['year']=papers['year'].astype("int")
papers['month']=papers['month'].astype("int")
#以下语法对部分样本可用，现在全样本存在错误，日期格式存在非法字符
#papers['date_time'] = pd.to_datetime(papers['year_period'])
#papers['year']=papers['date_time'].dt.year
#papers['month']=papers['date_time'].dt.month
papers=papers[papers["year"]<2018]
papers=papers[papers["year"]>2000]

#发表日期精确到月
papers['ym']=papers['year'].astype('str')+'-'+papers['month'].astype('str')+'-'+'01'
papers['ym']=pd.to_datetime(papers['ym']) 
#papers['year_period']=pd.to_datetime(papers['year_period']) #日期中存在非法字符，这个日期会报错
print(papers['ym'][0:5])

print("年份异常剔除后数量:"+str(len(papers))) #查看行*列数1541039

#起始页码处理和论文卷期内排序
papers['page_beg']=papers['page_range'].str.split('-',expand=True)[0]
papers['page_beg']=papers['page_beg'].str.split('+',expand=True)[0]
print(papers.shape)#查看行*列数
#page_beg存在4万个空值，暂不计算论文排序了，ps:stata计算排序要简单的多
#papers=papers[papers["page_beg"].astype('int')<10000] #存在一个异常起始页456789
#papers['mag_period']=papers['mag_name']+papers['year_period']
#papers['page_sort']=papers['page_beg'].groupby(papers['mag_period']).rank(ascending=0,method='dense')
#data['group_sort']=data['comment_num'].groupby(data['cate']).rank(ascending=0,method='dense')
#papers['abstract']=papers['abstract'].fillna('null') #将df中A列所有空值赋值为'null'

##分类号处理，只保留第一个字母
papers["class_num"] =papers['class_num'].str.replace("分类号：","")
papers['class_num']=papers['class_num'].str.replace('(\d+)',"") 
papers['class_num']=papers['class_num'].str.replace('.',"") 
papers['class_num']=papers['class_num'].str.replace('+',"")
papers['class_num']=papers['class_num'].str.replace('-',"")

#合作者提取，只以第一个“；”来分割，并保留第二个，从而得到第一作者之外的所有合作者
papers['coauthor']=papers['author'].str.split(';',n=1,expand=True)[1]  

#对学科分类进行重新调整，经济类和管理类合并，高校学报和综合社会科学合并
papers['mag_classn']=papers["mag_class"]
papers['mag_classn']=papers["mag_classn"].str.replace("经济学","经济与管理")
papers['mag_classn']=papers["mag_classn"].str.replace("管理学","经济与管理")
papers['mag_classn']=papers["mag_classn"].str.replace("高校学报","综合社会科学")

#数据保存
print(papers.shape)#查看行*列数
papers.to_csv(path+'cssci_clean.csv',encoding='utf8')

#数据太大，stata打不开，另保存一个只包含部分列的数据
papers_short=papers[['tlength','mag_name','mag_city_code','guanxi_drop','aunum','author_first','cited','download','fund01','fundn','page_num','year_period','year','month','kwnum','ablength','page_beg']] 
print(papers_short.shape)#查看行*列数
papers_short.to_csv(path+'cssci_short.csv',encoding='utf8')

#计算主题模型时，需要将标题、关键词和摘要合并
#papers['keyword']=papers['keyword'].fillna(";")
#papers['content']=papers['title']+";"+papers['keyword']+papers['abstract']
#papers=papers[papers['content'].str.len()>100]   #将标题+关键词+摘要少于100字的样本删除
#print("标题+关键词+摘要少于100字的样本删除后数量:"+str(len(papers))) #查看行*列数
#papers.to_csv(path+'cssci_clean1.csv',encoding='utf8')

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)


   Unnamed: 0  id                       title            author  \
0           0   1         中国式分权与新发展理念语境中的标尺竞争          刘珊珊; 马志远   
1           1   2  海洋生态补偿的制度建设与治理实践——基于国际比较视角  朱炜; 王乐锦; 王斌; 谈立群   
2           2   3    新常态下中国流动人口的居留意愿与家庭消费水平研究     李国正; 艾小青; 邬嘉迪   
3           3   4             提高最低工资会拉升产品价格吗?      徐建炜; 邹静娴; 毛捷   
4           4   5   多个大股东与企业融资约束——基于文本分析的经验证据  姜付秀; 王运通; 田园; 吴恺   

                                         author_unit mag_name year_period  \
0                                    上海财经大学公共经济与管理学院     管理世界  2017-12-15   
1                                       山东财经大学  燕山大学     管理世界  2017-12-15   
2                            北京工业大学经济与管理学院  华北水利水电大学     管理世界  2017-12-15   
3    北京师范大学经济与工商管理学院  人民大学国家发展与战略研究院  对外经济贸易大学国际经...     管理世界  2017-12-15   
4    中国人民大学商学院财务与金融系  武汉大学经济与管理学院博士后流动站  中国华融资产管理...     管理世界  2017-12-15   

   cited  download                         keyword  \
0      0       308      关键词：五大发展理念;\n中国式分权;\n标尺竞争;   
1      0       292       关

# 6.1.2：作者迁移

In [2]:
#最终目的：将科研人员在城市间的流动识别出来
#关键科学问题：同一个姓名的作者是真实流动还是仅仅是重名现象
#基本思路：利用共同合作者、共同的期刊学科分类、共同的论文中图分类号和发表时间间隔来判断是同一个人的流动还是仅仅是重名问题

#几个关键信息：去掉城市未识别和港澳台及国外机构后196万篇；
#剔除只出现于一个城市的作者后样本量:115万篇；姓名-单位聚合数37万对；姓名-城市聚合数29万对；
#一个城市内某姓名仅出现于一个单位占比84.6%。

#步骤1：将作者单位和所在城市进行识别
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
fcssci = open(path+"cssci_clean.csv",encoding='utf8')
papers= pd.read_csv(fcssci,header=0,sep=',')
print(papers.shape)#查看行*列数
#papers=papers[0:100]
print("剔除单位前数量:"+str(len(papers))) 
papers=papers.dropna(subset=["author_unit"]) #作者单位存在空值，删除
papers=papers[~papers['author_unit'].str.contains('编辑部')] 
papers=papers[~papers['author_unit'].str.contains('本刊记者')] 
papers=papers[~papers['author_unit'].str.contains('本刊特约记者')] 
papers=papers[~papers['author_unit'].str.contains('课题组')] 
print("剔除空值单位和其他异常单位后数量:"+str(len(papers))) #查看行*列数

#作者单位调整，一些作者单位不规则，比如：中国社会科学院，可能被写成：中国社科院
#这个调整当中存在很多陷阱，比如北大，可以认为是北京大学，但如果“北大”替换为“北京大学”，则“西北大学”也被替换为一个莫名其妙的单位
#中了无数次陷阱
papers['author_unit_adj']=papers['author_unit']
funitadj= open(path+"unit_name_adj.txt",encoding='utf8')
funitadj=funitadj.read()
for line in funitadj.split('\n'):
    for word in line.split()[1:]:
        if len(word)>0:
            papers["author_unit_adj"] =papers['author_unit_adj'].str.replace(word,line.split()[0])
        else:
            pass
papers['unitadj01']=(papers['author_unit_adj']==papers['author_unit']) 

print("作者单位，调整后：")
print(papers['author_unit_adj'][0:20]) 

#作者一级单位匹配提取：将”上海财经大学公共经济与管理学院”提取为“上海财经大学”
#第一种方法，先给定一个一级单位列表，但但数量越来越大，这个方法效率态度，运行速度太慢了
#funit= open(path+"unit_name.txt",encoding='utf8')
#funit=funit.read()
#units=funit.split('\n')
#papers['author_unit_short']=papers['author_unit_adj']
#papers['author_unit_short']=papers['author_unit_short'].str.replace('(\d+)',"") #去掉邮政编码、数字等。
#papers=papers.dropna(subset=["author_unit_short"]) 
#for unit in units:
#    papers['author_unit_short']=papers['author_unit_short'].str.replace('('+unit+'\S+)',unit) #正则表达式，下列式子存在问题
#    #papers['author_unit_short']=papers['author_unit_short'].str.replace('('unit\S+)',unit)  

#第二种方法:按单位关键词来匹配
kus=['大学','政府','检察院','法院','检察院','医院','图书馆','国务院','博物馆','博物院','档案馆','中学','党校','学校','统计局','气象局','电台','电视台','杂志社','报社','集团','公安局','委员会','基金会','协会','研究院','学院','研究所','学会','中心','师专','公司','银行','证券','统计局']
papers['author_unit_short']=papers['author_unit_adj']
papers['author_unit_short']=papers['author_unit_short'].str.replace('(\d+)',"") #去掉邮政编码、数字等。
papers=papers.dropna(subset=["author_unit_short"]) 
for ku in kus:
    papers['author_unit_short']=papers['author_unit_short'].str.replace('('+ku+'\S+)',ku) 
print("作者单位，缩写后：")
print(papers['author_unit_short'][0:20])  

#提取第一作者单位，分列:https://www.jianshu.com/p/4a2ecf65e3ea【分列】
papers['author_unit_first']=papers['author_unit_short'].str.split(expand=True)[0]
papers['author_unit_first']=papers['author_unit_first'].str.split(',',expand=True)[0]
papers['author_unit_first']=papers['author_unit_first'].str.split('!',expand=True)[0]
papers['author_unit_first']=papers['author_unit_first'].str.split('、',expand=True)[0]

#处理异常单位
papers["author_unit_first"] =papers['author_unit_first'].str.replace("《","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("》","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("（","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("）","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("\"","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("作者系","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("作者单位：","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace(".","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("[","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("]","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("中华人民共和国","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("中国人民解放军","")
papers["author_unit_first"] =papers['author_unit_first'].str.replace("中共","")

papers=papers[papers['author_unit_first'].str.len()>2]
print("剔除单位名称过短后数量:"+str(len(papers))) #查看行*列数
papers=papers.dropna(subset=["author_unit_first"]) 
print("作者第一单位，缩写后：")
print(papers['author_unit_first'][0:20])

#城市匹配，将第一作者单位与单位城市匹配
funit_city= open(path+"unit_city.csv",encoding='utf8')
unit_citys= pd.read_csv(funit_city,header=0,sep=',')
papers=pd.merge(papers,unit_citys,how='left')


#将匹配成功和未成功样本分隔开
papers1=papers.dropna(subset=["unit_city"]) 
papers2=papers[papers['unit_city'].isnull()]
#将匹配的城市名和代码暂时删除
papers2=papers2.drop(['unit_city','unit_city_code'],axis=1)

#稀有单位的进一步匹配
print("稀有单位匹配前数量:"+str(len(papers2))) #查看行*列数
print(papers2['author_unit_first'][0:20])
funit= open(path+"unit_name.txt",encoding='utf8')
funit=funit.read()
units=funit.split('\n')
for unit in units:
    if len(unit)>=2: #存在一些特殊错误符号，全给替换没了
        papers2['author_unit_first']=papers2['author_unit_first'].str.replace('('+unit+'\S+)',unit) #正则表达式，下列式子存在问题【unit当中不能存在()】

papers2=pd.merge(papers2,unit_citys,how='left')
print(papers2['author_unit_first'][0:20])
print(papers2['unit_city_code'][0:20])
papers= papers1.append(papers2)

papers=papers[['title','mag_name','mag_classn','mag_city','mag_city_code','author_first','author_unit_first','unit_city','unit_city_code','coauthor','ym','class_num']]
papers.to_csv(path+'cssci_city.csv',encoding='utf8')

papers=papers[papers['unit_city'].isnull()]
print("未匹配成功单位数量:"+str(len(papers))) #查看行*列数
#papers.drop_duplicates(subset=['author_unit_first'],keep='first',inplace=True) 
unit_rare=papers['author_unit_first']
unit_rare.to_csv(path+'unit_rare.csv',encoding='utf8',header=True) #18365个单位未识别，总样本中不足1%

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

(8666, 35)
剔除单位前数量:8666
剔除空值单位和其他异常单位后数量:8383
作者单位，调整后：
0                                       上海财经大学公共经济与管理学院
1                                          山东财经大学  燕山大学
2                               北京工业大学经济与管理学院  华北水利水电大学
3       北京师范大学经济与工商管理学院  人民大学国家发展与战略研究院  对外经济贸易大学国际经...
4       中国人民大学商学院财务与金融系  武汉大学经济与管理学院博士后流动站  中国华融资产管理...
5             中国科学院大学经济与管理学院  中国科学院大学经济与管理学院中国—丹麦科研教育中心
6                                                上海财经大学
7                               暨南大学产业经济研究院  北京大学光华管理学院
8                                          贵州财经大学工商管理学院
9                                             北京物资学院商学院
10                                            华中科技大学法学院
11                                           重庆大学公共管理学院
12                               安徽财贸职业学院大位会计学院  上海财经大学
13             南开大学经济学院、中国特色社会主义经济建设协同创新中心  清华大学创新发展研究院
14              上海财经大学经济学院  上海财经大学数理经济学重点实验室  美国里海大学经济系
15      中国女企业家协会  中国企业家调查系统  北京卓越企业家成长研究基金会  国务院发展研究...
16                                             兰

In [3]:
#步骤2：论文按照作者-单位进行聚合
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
fcssci = open(path+"cssci_city.csv",encoding='utf8')
papers= pd.read_csv(fcssci,header=0,sep=',')
#papers=papers[0:1000]

print("原始数量:"+str(len(papers))) #查看行*列数
papers=papers.dropna(subset=["unit_city_code"])  #删除城市信息没有识别的样本【近2万多个未识别】
print("删除城市未识别后数量:"+str(len(papers))) #查看行*列数
papers=papers[papers["unit_city_code"]<7100]  #7100台湾省；8100香港；8200澳门；9000国外
print("删除港澳台及国外后单位数量:"+str(len(papers))) #查看行*列数
papers=papers[~papers['author_first'].str.contains('课题组')]
print("删除课题组为单位署名的:"+str(len(papers))) #查看行*列数

#将只在一个城市出现过的样本剔除，剔除样本不足一半，说明重名问题非常严重
#计算A列在B列中究竟对应几个不同的值，相信这里应该有非常简单的方法，但没找到
#剔除原理：按名分组后计算城市代码的最大值和最小值，两者如果一样说明，说明这个名字只在一个城市出现过，剔除
group1= papers.groupby(["author_first"])
temp=pd.DataFrame(columns=['city_code_max',"city_code_min"])
temp['city_code_max']=group1['unit_city_code'].max()
temp['city_code_min']=group1['unit_city_code'].min()
temp['city_code1']=(temp['city_code_max']>temp['city_code_min'])
temp.to_csv(path+'temp.csv',encoding='utf8')
ftemp = open(path+"temp.csv",encoding='utf8')
temp= pd.read_csv(ftemp,header=0,sep=',')
papers=pd.merge(papers,temp,on=["author_first"],how='left')
papers=papers[papers['city_code1'].astype('str')=="True"]
print("剔除只出现于一个城市的作者后样本量:"+str(len(papers))) #查看行*列数
print(papers[0:20])

#按照作者-单位聚合数据
papers=papers[['title','author_first','author_unit_first','ym','unit_city_code','coauthor','mag_classn','class_num']]
papers['class_num']=papers['class_num'].str.replace('-',"")
papers['ym']=pd.to_datetime(papers['ym'])
#聚合论文的中图分类号、学科领和合作者，先增加一个";",以便后面再重新展开
papers['class_num']=papers['class_num'].astype('str')+";"
papers['mag_classn']=papers['mag_classn'].astype('str')+";"
papers['coauthor']=papers['coauthor'].astype('str')+";"
papers['ym']=papers['ym'].astype('str')+";"
print(papers[0:10])
group2= papers.groupby(["author_first",'author_unit_first','unit_city_code'])
author_units=pd.DataFrame(columns=["ym","coauthor","class_num",'mag_classn'])
author_units['ym']=group2['ym'].sum()
author_units['coauthor']=group2['coauthor'].sum()
author_units['class_num']=group2['class_num'].sum()
author_units['mag_classn']=group2['mag_classn'].sum()
author_units["coauthor"] =author_units['coauthor'].str.replace("nan;","")
author_units.to_csv(path+'author_units.csv',encoding='utf8')
f1 = open(path+"author_units.csv",encoding='utf8')
author_units= pd.read_csv(f1,header=0,sep=',')
print("作者-单位聚合后样本量:"+str(len(author_units))) #查看行*列数

#计算一个“作者-单位”对应的学科类别数和中图分类号
#先编写一个小程序，然后用map、apply等调用，可以对dataframe进行整列结算，大大提高效率
author_units['class_num_num']=author_units["class_num"].map(lambda x:len(set(x[0:-1].split(';'))))   
author_units['mag_classn_num']=author_units["mag_classn"].map(lambda x:len(set(x[0:-1].split(';')))) 

#计算城市内部某姓名对应的单位数量
author_units['author_unit_first']=author_units['author_unit_first'].astype('str')+";"
group3= author_units.groupby(["author_first",'unit_city_code'])
city_unitn=pd.DataFrame(columns=['author_first','unit_city_code','city_unitn'])
city_unitn['city_unitn']=group3['author_unit_first'].count()
city_unitn['city_units']=group3['author_unit_first'].sum()
city_unitn.to_csv(path+'author_units.csv',encoding='utf8')
f2 = open(path+"author_units.csv",encoding='utf8')
city_unitn= pd.read_csv(f2,header=0,sep=',')
author_units=pd.merge(author_units,city_unitn,on=["author_first",'unit_city_code'],how='left')
author_units.to_csv(path+'author_units.csv',encoding='utf8',index=False)
print(author_units.shape)

#将某单位内对应三个以上中图分类号且三个期刊学科领域的姓名删除
author_units=author_units[~((author_units['class_num_num']>3) & (author_units['mag_classn_num']>3)) ]  
print("某单位内对应三个以上中图分类号且三个期刊学科领域的姓名删除:"+str(len(author_units))) #查看行数

author_citys_oneunit=author_units[author_units['city_unitn']==1]
print("一个城市内某姓名仅出现于一家单位的数量:"+str(len(author_citys_oneunit))) #查看行数
author_citys_oneunit.to_csv(path+'author_citys_oneunit.csv',encoding='utf8',index=False)
print(author_citys_oneunit.head())
#查看一个城市内出现于多个单位的姓名数量
author_citys_moreunits=author_units[author_units['city_unitn']>1]
print("一个城市内某姓名出现在多家单位的数量:"+str(len(author_citys_moreunits))) #查看行数

#删除一个城市内部对应单位超过5家的超常见姓名
author_citys_moreunits=author_citys_moreunits[author_citys_moreunits['city_unitn']<=5]
print("删除一个城市内部对应单位超过5家的超常见姓名后的数量:"+str(len(author_citys_moreunits))) #查看行数
author_citys_moreunits.to_csv(path+'author_citys_moreunits.csv',encoding='utf8',index=False)
print(author_citys_moreunits.head())
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

原始数量:8375
删除城市未识别后数量:7509
删除港澳台及国外后单位数量:7279
删除课题组为单位署名的:7225
剔除只出现于一个城市的作者后样本量:1029
     Unnamed: 0                                         title mag_name  \
6             6                   思维定势如何影响创意质量——基于“众包”平台的实证研究     管理世界   
7             7                         内部员工优势——来自企业人事数据的实证证据     管理世界   
13           13     经济发展中的技术创新、技术溢出与行业收入差距演变——对U型假说的理论解释与实证检验     管理世界   
17           17  社会责任信息披露的“掩饰效应”和上市公司崩盘风险——来自中国股票市场的DID-PSM分析     管理世界   
21           21                            政治激励、资本监管与地方银行信贷投放     管理世界   
29           29                              社会信任是否会提升企业并购绩效?     管理世界   
32           32                             经济学研究中“数学滥用”现象及反思     管理世界   
34           34                            中国劳动力市场所有制分割的根源与表现     管理世界   
38           38                          从自利到德行:商业组织的制度逻辑变革研究     管理世界   
63           63                 政府支持型自组织构建——基于深圳食品安全社会共治的案例研究     管理世界   
82           83                空气污染对企业库存的影响——基于我国制造业企业数据的实证研究     管理世界   
84           85            

73             NaN               NaN           2  中国社会科学院;对外经济贸易大学;  
0


In [4]:
#步骤3：一个城市内部对应多家单位的姓名配对

import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_citys_moreunits.csv",encoding='utf8')
au_units= pd.read_csv(f1,header=0,sep=',')
print(au_units.shape)
#au_units=au_units[0:2000]

print(au_units.shape)
print(au_units[0:10])
au_units.sort_values(by=["author_first",'unit_city_code'],inplace=True)

au_unitsn=pd.DataFrame(columns=('author_first','unit_city_code','author_unit_first1','ym1','class_num1','mag_classn1','coauthor1',
                                'author_unit_first2','ym2','class_num2','mag_classn2','coauthor2'),index=range(0,100000))
n=0
i=0 
while i<len(au_units):
    j=i+1
    while j<len(au_units):
        if (au_units['author_first'][j]==au_units['author_first'][i]) & (au_units['unit_city_code'][j]==au_units['unit_city_code'][i]):
            au_unitsn['author_first'][n]=au_units['author_first'][i]
            au_unitsn['unit_city_code'][n]=au_units['unit_city_code'][i]
            au_unitsn['author_unit_first1'][n]=au_units['author_unit_first'][i]
            au_unitsn['ym1'][n]=au_units['ym'][i]
            au_unitsn['class_num1'][n]=au_units['class_num'][i]
            au_unitsn['mag_classn1'][n]=au_units['mag_classn'][i]
            au_unitsn['coauthor1'][n]=au_units['coauthor'][i]
            au_unitsn['author_unit_first2'][n]=au_units['author_unit_first'][j]
            au_unitsn['ym2'][n]=au_units['ym'][j]
            au_unitsn['class_num2'][n]=au_units['class_num'][j]
            au_unitsn['mag_classn2'][n]=au_units['mag_classn'][j] 
            au_unitsn['coauthor2'][n]=au_units['coauthor'][j]
            n=n+1
            j=j+1
        else:
            break
    print(i)        
    i=j

print(au_unitsn[0:5])

print(au_unitsn.shape)
au_unitsn.dropna(subset=["unit_city_code"],inplace=True)
print(au_unitsn.shape)
au_unitsn.to_csv(path+'author_unit_and_unit.csv',encoding='utf8',index=False)

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

(67, 13)
(67, 13)
  author_first author_unit_first  unit_city_code  \
0          何雄浪           西南民族大学;          5101.0   
1          何雄浪           西南财经大学;          5101.0   
2           刘斌           中国人民银行;          1100.0   
3           刘斌         对外经济贸易大学;          1100.0   
4           刘青          中国社会科学院;          1100.0   
5           刘青         对外经济贸易大学;          1100.0   
6           刘飞           西安交通大学;          6101.0   
7           刘飞           西安邮电大学;          6101.0   
8           张伟             贵州大学;          5201.0   
9           张伟           贵州财经大学;          5201.0   

                                             ym            coauthor class_num  \
0                                   2015-04-01;                 NaN      X;F;   
1                                   2013-04-01;                 赵峰;      F;F;   
2  2003-09-01;2002-07-01;2001-07-01;2001-10-01;                张怀清;  F;F;F;F;   
3                        2016-03-01;2016-09-01;   魏倩; 吕越; 祝坤福; 王乃嘉;    F;F;F;   
4   

In [5]:
#步骤4：计算一个城市多家单位同一个姓名拥有的期刊学科类别数和中图分类号交集次数

import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_unit_and_unit.csv",encoding='utf8')
au_unitsn= pd.read_csv(f1,header=0,sep=',')
print(au_unitsn.shape)

#就算某姓名城市内多单位之间的中图分类号交集个数
def f1(x):
    return len(set(x['class_num1'][0:-1].split(';')) & set(x['class_num2'][0:-1].split(';'))) 
au_unitsn['class_overlap']=au_unitsn.apply(f1, axis=1)

#计算某姓名在城市内多单位间的期刊领域交集个数
def f2(x):
    return len(set(x['mag_classn1'][0:-1].split(';')) & set(x['mag_classn2'][0:-1].split(';')))
au_unitsn['mag_overlap']=au_unitsn.apply(f2, axis=1)

#计算某姓名在城市内多单位间之间共同合作者个数
au_unitsn['coauthor1n']=au_unitsn['coauthor1'].fillna(';')
au_unitsn['coauthor2n']=au_unitsn['coauthor2'].fillna(';')
def f3(x):
    if (pd.isnull(x['coauthor1'])) | (pd.isnull(x['coauthor2'])):
        return 0
    else:
        return len(set(x['coauthor1n'][0:-1].split(';')) & set(x['coauthor2n'][0:-1].split(';'))) 
au_unitsn['coauthor_overlap']=au_unitsn.apply(f3, axis=1)

#计算某姓名在城市内多单位之间“流动'的次数
def f4(x):
    time=x['ym1']+x['ym2']
    time=time[0:-1].split(';')
    unit=(str(x['author_unit_first1'])+";")*len(x['ym1'][0:-1].split(';'))+(str(x['author_unit_first2'])+";")*len(x['ym2'][0:-1].split(';'))
    unit=unit[0:-1].split(';')
    unit_num=(str(1)+';')*len(x['ym1'][0:-1].split(';'))+(str(2)+';')*len(x['ym2'][0:-1].split(';'))
    unit_num=unit_num[0:-1].split(';')
    df_temp= pd.DataFrame({'t':time,'u':unit_num})
    df_temp['t']=pd.to_datetime(df_temp['t'])
    df_temp.sort_values(by=["t"],inplace=True)
    df_temp['diff']=df_temp['u'].astype(float).diff()
    return ((df_temp['diff'] != 0) & (df_temp['diff'].astype(str) != 'nan')) .astype(int).sum()
au_unitsn['move_time']=au_unitsn.apply(f4, axis=1)
    
print(au_unitsn[0:20])        

au_unitsn=au_unitsn[['author_first','unit_city_code','author_unit_first1','ym1','coauthor1','class_num1','mag_classn1','author_unit_first2','ym2','coauthor2','class_num2','mag_classn2','move_time','class_overlap','mag_overlap','coauthor_overlap']]
print(au_unitsn.shape)
print(au_unitsn[0:20])
au_unitsn.to_csv(path+'author_unit_and_unit_classnum.csv',encoding='utf8',index=False)                
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)                

(34, 12)
   author_first  unit_city_code author_unit_first1  \
0           何雄浪          5101.0            西南民族大学;   
1            刘斌          1100.0            中国人民银行;   
2            刘青          1100.0           中国社会科学院;   
3            刘飞          6101.0            西安交通大学;   
4            张伟          5201.0              贵州大学;   
5            张兵          3201.0            南京农业大学;   
6            张勇          1100.0            中国人民大学;   
7           张卫国          3701.0              山东大学;   
8           张卫国          5000.0              西南大学;   
9            张强          1100.0            中国人民大学;   
10          张海洋          3301.0              浙江大学;   
11          李君华          4201.0            华中科技大学;   
12          李宏瑾          1100.0            中国人民银行;   
13          李尚骜          4201.0            华中科技大学;   
14          李建军          1100.0            中央财经大学;   
15           李斌          1100.0            中国人民银行;   
16           李楠          1100.0            中国人民大学;   
17           李楠    

In [6]:
#步骤5：城市内多单位姓名的识别及姓名-城市的聚合

import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_unit_and_unit_classnum.csv",encoding='utf8')
au_unitsn= pd.read_csv(f1,header=0,sep=',')
print(au_unitsn.head())

#是否重名做标签，标签规则参考下文城市间重名的识别，有所简略
def f(x):
    if x['coauthor_overlap']>0:
        return 1
    elif x['move_time']>2:
        return 0
    elif  (x['class_overlap']>0) | (x['mag_overlap']>0):
        return 1
    else:
        return 0
au_unitsn['sameone']=au_unitsn.apply(f, axis=1)

#城市内多单位姓名为重名的，则将其相关特征合并，否则提取第一个单位的信息，然后第二个单位信息识别为另外一人。
#理论上可以多次循环，但实际上不值得

#非重名单位信息合并
au_unitsn1=au_unitsn[au_unitsn['sameone'].astype(str)=="1"] 
au_unitsn1['author_unit_first1']=au_unitsn1['author_unit_first1'].astype('str')+";"
au_unitsn1['coauthor1']=au_unitsn1['coauthor1'].fillna(';')
au_unitsn1['coauthor2']=au_unitsn1['coauthor2'].fillna(';')
group1=au_unitsn1.groupby(["author_first",'unit_city_code'])
au_citysn1=pd.DataFrame(columns=['ym','class_num','mag_classn'])
au_citysn1['ym']=group1['ym2'].sum()+group1['ym1'].max()
au_citysn1['city_unitn']=group1['author_unit_first2'].count()+1
au_citysn1['city_units']=group1['author_unit_first2'].sum()+group1['author_unit_first1'].max()
au_citysn1['coauthor']=group1['coauthor2'].sum()+group1['coauthor1'].max()
au_citysn1['class_num']=group1['class_num2'].sum()+group1['class_num1'].max()
au_citysn1['mag_classn']=group1['mag_classn2'].sum()+group1['mag_classn1'].max()
au_citysn1['author_first']=group1['author_first'].max()
au_citysn1['unit_city_code']=group1['unit_city_code'].max()

au_citysn1.to_csv(path+'author_citys_moreunits1.csv',encoding='utf8')
f1 = open(path+"author_citys_moreunits1.csv",encoding='utf8')
au_citysn1= pd.read_csv(f1,header=0,sep=',')
au_citysn1.to_csv(path+'author_citys_moreunits1.csv',encoding='utf8',index=False)
print(au_citysn1.shape)
print(au_citysn1.coauthor[0:20])

#重名单位分开提取，左半边
au_unitsn2=au_unitsn[au_unitsn['sameone'].astype(str)=="0"] 
au_citysn2=au_unitsn2[['author_first','author_unit_first1','unit_city_code','ym1','class_num1','mag_classn1','coauthor1','sameone']]
au_unitsn2['city_unitn']=1
au_unitsn2['city_unitns']=au_unitsn2['author_unit_first1']
au_citysn2.rename(columns={'ym1':'ym','author_unit_first1':'author_unit_first', 'class_num1':'class_num','mag_classn1':'mag_classn','coauthor1':'coauthor'}, inplace = True)
au_citysn2.to_csv(path+'author_citys_moreunits2.csv',encoding='utf8',index=False)
print(au_citysn2.shape)


#重名单位分开提取，右半边
au_unitsn3=au_unitsn[au_unitsn['sameone'].astype(str)=="0"] 
au_unitsn3=au_unitsn3[['author_first','author_unit_first2','unit_city_code','ym2','class_num2','mag_classn2','coauthor2','sameone']]
au_unitsn3['author_first']=au_unitsn3['author_first'].astype(str)+'_2'
au_unitsn3.rename(columns={'ym2':'ym', 'author_unit_first2':'author_unit_first','class_num2':'class_num','mag_classn2':'mag_classn','coauthor2':'coauthor'}, inplace = True)
au_unitsn3.drop_duplicates(subset=['author_first','author_unit_first','unit_city_code'],keep='first',inplace=True) 
au_unitsn3.to_csv(path+'author_moreunits3.csv',encoding='utf8',index=False)
print(au_unitsn3.shape)


#把au_unitsn3的姓名在作者-单位聚合基础上聚合为作者-城市
au_unitsn3['author_unit_first']=au_unitsn3['author_unit_first'].astype('str')+";"
au_unitsn3['coauthor']=au_unitsn3['coauthor'].fillna(';')
group3=au_unitsn3.groupby(["author_first",'unit_city_code'])
au_citysn3=pd.DataFrame(columns=['ym','class_num','mag_classn'])
au_citysn3['ym']=group3['ym'].sum()
au_citysn3['city_unitn']=group3['author_unit_first'].count()
au_citysn3['city_units']=group3['author_unit_first'].sum()
au_citysn3['coauthor']=group3['coauthor'].sum()
au_citysn3['class_num']=group3['class_num'].sum()
au_citysn3['mag_classn']=group3['mag_classn'].sum()

au_citysn3.to_csv(path+'author_citys_moreunits3.csv',encoding='utf8')
f3 = open(path+"author_citys_moreunits3.csv",encoding='utf8')
au_citysn3= pd.read_csv(f3,header=0,sep=',')
au_citysn3.to_csv(path+'author_citys_moreunits3.csv',encoding='utf8',index=False)

print(au_citysn3.shape)
print(au_citysn3.head())

#姓名-城市聚合后的结果如下
f2 = open(path+"author_citys_oneunit.csv",encoding='utf8')
au_citys= pd.read_csv(f2,header=0,sep=',')
print(au_citys.shape) 
au_citys=au_citys.append(au_citysn1)
print(au_citys.shape) 
au_citys=au_citys.append(au_citysn2)
print(au_citys.shape) 
au_citys=au_citys.append(au_citysn3)
print(au_citys.shape)

#重新算一个“作者-城市”对应的学科类别数和中图分类号
au_citys['class_num_num']=au_citys["class_num"].map(lambda x:len(set(x[0:-1].split(';'))))   
au_citys['mag_classn_num']=au_citys["mag_classn"].map(lambda x:len(set(x[0:-1].split(';'))))

#再次将某城市内对应三个以上中图分类号且三个期刊学科领域的姓名删除
au_citys=au_citys[~((au_citys['class_num_num']>3) & (au_citys['mag_classn_num']>3)) ]  
print(au_citys.shape)

au_citys=au_citys[['author_first','author_unit_first','unit_city_code','city_unitn','city_units','class_num','class_num_num','coauthor','mag_classn',
                          'mag_classn_num','ym']]

au_citys["author_first0"] =au_citys['author_first'].str.replace("_2","")
au_citys.sort_values(by=["author_first0"],inplace=True)
au_citys.to_csv(path+'author_citys.csv',encoding='utf8',index=False)

print(au_citys.tail())
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

  author_first  unit_city_code author_unit_first1  \
0          何雄浪          5101.0            西南民族大学;   
1           刘斌          1100.0            中国人民银行;   
2           刘青          1100.0           中国社会科学院;   
3           刘飞          6101.0            西安交通大学;   
4           张伟          5201.0              贵州大学;   

                                            ym1 coauthor1 class_num1  \
0                                   2015-04-01;       NaN       X;F;   
1  2003-09-01;2002-07-01;2001-07-01;2001-10-01;      张怀清;   F;F;F;F;   
2                                   2012-07-01;       NaN         F;   
3                                   2011-08-01;       NaN         F;   
4                                   2011-10-01;      吴文元;       F;F;   

                mag_classn1 author_unit_first2                     ym2  \
0                    经济与管理;            西南财经大学;             2013-04-01;   
1  经济与管理;经济与管理;经济与管理;经济与管理;          对外经济贸易大学;  2016-03-01;2016-09-01;   
2                    经济与管理

D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [7]:
#补充：另一种对城市内部多单位作者进行识别（剔除）的方法
#直接在姓名-单位基础上聚合为姓名-城市，然后根据姓名对应的单位数和期刊学科分布、中图分类号分布，来进行剔除某城市内部有重名嫌疑的姓名
#剔除规则：#(a)如果某姓名在某城市内的4个单位及以上单位出现过，但全部将其剔除。
#(b)如果某姓名在某城市内的3个单位出现过，但其对应的期刊学科分类大于1个，且中图分类号大于1个，则剔除。
#(c)如果某姓名在某城市内的2个单位出现过，但其对应的期刊学科分类大于2个，且中图分类号大于2个，则剔除。
#(d)如果某姓名只在某城市内的1个单位出现过，但其对应的期刊学科分类大于3个，且中图分类号大于3个，则剔除。
#我自己的论文就存在三个中图分类号
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_units.csv",encoding='utf8')
author_units=pd.read_csv(f1,header=0,sep=',')

#在作者-单位聚合基础上，直接聚合为作者-城市
author_units['author_unit_first']=author_units['author_unit_first'].astype('str')+";"
author_units['coauthor']=author_units['coauthor'].fillna('nan')
group3= author_units.groupby(["author_first",'unit_city_code'])
author_citys=pd.DataFrame(columns=['ym','city_unitn',"city_units",'class_num','mag_classn'])
author_citys['ym']=group3['ym'].sum()
author_citys['city_unitn']=group3['author_unit_first'].count()
author_citys['city_units']=group3['author_unit_first'].sum()
author_citys['coauthor']=group3['coauthor'].sum()
author_citys['class_num']=group3['class_num'].sum()
author_citys['mag_classn']=group3['mag_classn'].sum()
author_citys['author_first']=group3['author_first'].max()
author_citys['unit_city_code']=group3['unit_city_code'].max()
print("出现于多个城市的姓名数量:"+str(len(author_citys)))

#编写一个小程序，然后用map、apply等调用，可以对dataframe进行整列结算，大大提高效率
author_citys['class_num_num']=author_citys["class_num"].map(lambda x:len(set(x[0:-1].split(';'))))   
author_citys['mag_classn_num']=author_citys["mag_classn"].map(lambda x:len(set(x[0:-1].split(';')))) 

#原使用循环的方法，计算非常耗时，map能够对整列进行计算
#author_citys['class_num_num']=0
#author_citys['mag_num_num']=0
#for i in range(0,len(author_citys)):
#    temp1=author_citys['class_num'][i][0:-1].split(';')
#    temp2=author_citys['mag_classn'][i][0:-1].split(';')
#    author_citys['class_num_num'][i]=len(set(temp1))
#    author_citys['mag_classn_num'][i]=len(set(temp2))
#    print(i)

author_citys=author_citys[author_citys['city_unitn']<4] 
print(author_citys.shape)  
author_citys=author_citys[~(((author_citys['class_num_num']>1) & (author_citys['mag_classn_num']>1)) & (author_citys['city_unitn']==3))] 
print(author_citys.shape)  
author_citys=author_citys[~(((author_citys['class_num_num']>2) & (author_citys['mag_classn_num']>2)) & (author_citys['city_unitn']==2))] 
print(author_citys.shape) 
author_citys=author_citys[~(((author_citys['class_num_num']>3) & (author_citys['mag_classn_num']>3)) & (author_citys['city_unitn']==1))]  
print(author_citys.shape)    

author_citys.sort_values(by=["author_first"],inplace=True)

author_citys.to_csv(path+'author_citys_anothermethod.csv',encoding='utf8')                                   
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

出现于多个城市的姓名数量:609
(609, 10)
(609, 10)
(609, 10)
(609, 10)
0


In [2]:
#学习样本
import pandas as pd
df = pd.DataFrame({'a':['a;b;a;c;','a;c;c;d;e;'],
                   'b':['e;d;f','a;b;d']})
print(df)
i=0 
df['c']=0 
while i<len(df):
    temp=df['a'][i][0:-1].split(';')
    temp=len(set(temp))
    df['c'][i]=temp
    i=i+1
print(df)

import pandas as pd
df2 = pd.DataFrame({'a':['a;b;a;c;','a;c;c;d;e;'],
                   'b':['e;d;f','a;b;d']})
print(df2)
df2['c']=df2["a"].map(lambda x:len(set(x[0:-1].split(';'))))
print(df2)

            a      b
0    a;b;a;c;  e;d;f
1  a;c;c;d;e;  a;b;d
            a      b  c
0    a;b;a;c;  e;d;f  3
1  a;c;c;d;e;  a;b;d  4
            a      b
0    a;b;a;c;  e;d;f
1  a;c;c;d;e;  a;b;d
            a      b  c
0    a;b;a;c;  e;d;f  3
1  a;c;c;d;e;  a;b;d  4


D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
#步骤6：城市间姓名配对
#暂不考虑重名问题，将所有可能的配对都识别出来
#到底city_code1流向city_code2还是反过来，暂不考虑

import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_citys.csv",encoding='utf8')
au_city= pd.read_csv(f1,header=0,sep=',')
print(au_city.shape)
print(au_city.head())  
print(au_city.tail())  

#au_city=au_city[0:100]

au_cityn=pd.DataFrame(columns=('author_first','city_code1','ym1','city_unitn1','city_units1','class_num1','mag_classn1','class_num_num1','mag_classn_num1','coauthor1','city_code2','ym2','city_unitn2','city_units2','class_num2','mag_classn2','class_num_num2','mag_classn_num2','coauthor2'),index=range(0,1000000))

n=0
i=0 
while i<len(au_city):
    j=i+1
    while j<len(au_city):
        if au_city['author_first0'][j]==au_city['author_first0'][i]:
            if au_city['unit_city_code'][j]==au_city['unit_city_code'][i]:
                j=j+1
            else:
                au_cityn['author_first'][n]=au_city['author_first'][i]
                au_cityn['city_code1'][n]=au_city['unit_city_code'][i]
                au_cityn['ym1'][n]=au_city['ym'][i]
                au_cityn['city_unitn1'][n]=au_city['city_unitn'][i]
                au_cityn['city_units1'][n]=au_city['city_units'][i]
                au_cityn['class_num1'][n]=au_city['class_num'][i]
                au_cityn['mag_classn1'][n]=au_city['mag_classn'][i]
                au_cityn['class_num_num1'][n]=au_city['class_num_num'][i]
                au_cityn['mag_classn_num1'][n]=au_city['mag_classn_num'][i]
                au_cityn['coauthor1'][n]=au_city['coauthor'][i]
                au_cityn['city_code2'][n]=au_city['unit_city_code'][j]
                au_cityn['ym2'][n]=au_city['ym'][j]
                au_cityn['city_unitn2'][n]=au_city['city_unitn'][j]
                au_cityn['city_units2'][n]=au_city['city_units'][j]
                au_cityn['class_num2'][n]=au_city['class_num'][j]
                au_cityn['mag_classn2'][n]=au_city['mag_classn'][j] 
                au_cityn['class_num_num2'][n]=au_city['class_num_num'][j]
                au_cityn['mag_classn_num2'][n]=au_city['mag_classn_num'][j]
                au_cityn['coauthor2'][n]=au_city['coauthor'][j]
                n=n+1
                j=j+1
        else:
            break
    print(i)        
    i=i+1

print(au_cityn[0:5])

print(au_cityn.shape)
au_cityn.dropna(subset=["city_code2"],inplace=True)
print(au_cityn.shape)
au_cityn.to_csv(path+'author_city_and_city.csv',encoding='utf8',index=False)

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

(609, 12)
  author_first author_unit_first  unit_city_code  city_unitn city_units  \
0          万广华           云南财经大学;          5301.0         1.0    云南财经大学;   
1          万广华         西北农林科技大学;          6104.0         1.0  西北农林科技大学;   
2           云鹤             中山大学;          4401.0         1.0      中山大学;   
3           云鹤           浙江理工大学;          3301.0         1.0    浙江理工大学;   
4           付强             山东大学;          3701.0         1.0      山东大学;   

                class_num  class_num_num                          coauthor  \
0  F;F;F;F;F;D;D;F;F;F;F;              2   张茵; M.S.Qureshi; 伏润民; 吴一平; 张藕香;   
1                    F;F;              1                               张茵;   
2                    F;F;              1                           刘涛; 舒元;   
3            F;F;F;F;F;F;              1                  舒元; 胡剑锋; 吕品; 舒元;   
4                      F;              1                               NaN   

                             mag_classn  mag_classn_num  \
0  经济与管理;经济

In [10]:
#步骤7：计算两个城市内同以个姓名拥有的期刊学科类别数和中图分类号交集次数

import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_city_and_city.csv",encoding='utf8')
au_cityn= pd.read_csv(f1,header=0,sep=',')
print(au_cityn.shape)
#au_cityn=au_cityn[0:145]

def f1(x):
    return len(set(x['class_num1'][0:-1].split(';')) & set(x['class_num2'][0:-1].split(';'))) 
au_cityn['class_overlap']=au_cityn.apply(f1, axis=1)

#计算某姓名在城市对之间期刊领域交集个数
def f2(x):
    return len(set(x['mag_classn1'][0:-1].split(';')) & set(x['mag_classn2'][0:-1].split(';')))
au_cityn['mag_overlap']=au_cityn.apply(f2, axis=1)

#计算某姓名在城市对之间共同合作者个数
au_cityn['coauthor1n']=au_cityn['coauthor1'].replace(';','')
au_cityn['coauthor2n']=au_cityn['coauthor2'].replace(';','')
def f3(x):
    if (pd.isnull(x['coauthor1n'])) | (pd.isnull(x['coauthor2n'])):
        return 0
    else:
        return len(set(x['coauthor1'][0:-1].split(';')) & set(x['coauthor2'][0:-1].split(';'))) 
au_cityn['coauthor_overlap']=au_cityn.apply(f3, axis=1)

#计算某姓名在城市对之间“流动'的次数
def f4(x):
    time=x['ym1']+x['ym2']
    time=time[0:-1].split(';')
    city=(str(x['city_code1'])+";")*len(x['ym1'][0:-1].split(';'))+(str(x['city_code2'])+";")*len(x['ym2'][0:-1].split(';'))
    city=city[0:-1].split(';')
    df_temp= pd.DataFrame({'t':time,'c':city})
    df_temp['t']=pd.to_datetime(df_temp['t'])
    df_temp.sort_values(by=["t"],inplace=True)
    df_temp['diff']=df_temp['c'].astype(float).diff()
    return ((df_temp['diff'] != 0) & (df_temp['diff'].astype(str) != 'nan')) .astype(int).sum()
au_cityn['move_time']=au_cityn.apply(f4, axis=1)
    
#au_cityn['class_overlap']=0
#au_cityn['mag_overlap']=0
#au_cityn['coauthorlap']=0
#au_cityn['coauthor1n']=au_cityn['coauthor1'].fillna(';')
#au_cityn['coauthor2n']=au_cityn['coauthor2'].fillna(';')
#for i in range(0,len(au_cityn)):
#    class_num1=au_cityn['class_num1'][i][0:-1].split(';')
#    class_num2=au_cityn['class_num2'][i][0:-1].split(';')
#    mag_class1=au_cityn['mag_classn1'][i][0:-1].split(';')
#    mag_class2=au_cityn['mag_classn2'][i][0:-1].split(';')
#    coauthor1=au_cityn['coauthor1n'][i][0:-1].split(';')
#    coauthor2=au_cityn['coauthor2n'][i][0:-1].split(';')
#    au_cityn['class_overlap'][i]=len(set(class_num1) & set(class_num2))
#    au_cityn['mag_overlap'][i]=len(set(mag_class1) & set(mag_class2))
#    if (au_cityn['coauthor1'].astype(str)[i]=="nan") | (au_cityn['coauthor2'].astype(str)[i]=="nan"):
#        au_cityn['coauthorlap'][i]=0
#    else:
#        au_cityn['coauthorlap'][i]=len(set(coauthor1) & set(coauthor2))
#    print(i)
print(au_cityn[0:20])        

au_cityn=au_cityn[['author_first','city_code1','ym1','coauthor1','class_num1','mag_classn1','city_code2','ym2','coauthor2','class_num2','mag_classn2','move_time','class_overlap','mag_overlap','coauthor_overlap']]
print(au_cityn.shape)
print(au_cityn[0:20])
au_cityn.to_csv(path+'author_city_and_city_classnum_anothermethod.csv',encoding='utf8',index=False)                
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)                

(357, 19)
   author_first  city_code1  \
0           万广华      5301.0   
1            云鹤      4401.0   
2            付强      3701.0   
3           付文林      3100.0   
4            代谦      3502.0   
5           任寿根      3601.0   
6           何启志      3201.0   
7           何德旭      1100.0   
8           何德旭      1100.0   
9           何德旭      1401.0   
10          何自力      4401.0   
11          何诚颖      4403.0   
12          何雄浪      5101.0   
13           何青      1100.0   
14           余翔      1100.0   
15           俞乔      3100.0   
16          俞红海      3100.0   
17           倪馨      1100.0   
18          冯旭南      3100.0   
19           刘冲      3100.0   

                                                  ym1  city_unitn1  \
0   2013-05-01;2008-12-01;2008-04-01;2012-04-01;20...          1.0   
1                                         2004-07-01;          1.0   
2                                         2017-03-01;          1.0   
3                              2014-05-01;2012-05-01;     

In [11]:
#步骤8：城市级科研人才流动识别，暂不考虑重名
#将a到b，b到a识别成两个（a和b多次交替出现的，不识别，只标注出move_time）
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_city_and_city_classnum_anothermethod.csv",encoding='utf8')
au_cityn= pd.read_csv(f1,header=0,sep=',')
print(au_cityn.shape)
print(au_cityn[0:20])

au_cityn['ymmin1']=0
au_cityn['ymmax1']=0
au_cityn['ymmin2']=0
au_cityn['ymmax2']=0
au_cityn['ymmin_new']=0
au_cityn['ymmax_new']=0
def f(x):
    x['ymmin1']=min(x['ym1'][0:-1].split(';')) 
    x['ymmax1']=max(x['ym1'][0:-1].split(';'))
    x['ymmin2']=min(x['ym2'][0:-1].split(';')) 
    x['ymmax2']=max(x['ym2'][0:-1].split(';')) 
    if x['move_time']==1:
        if x['ymmax2']<x['ymmin1']:
            x['city_code1'],x['city_code2']=x['city_code2'],x['city_code1']
            x['ym1'],x['ym2']=x['ym2'],x['ym1']
            x['coauthor1'],x['coauthor2']=x['coauthor2'],x['coauthor1']
            x['class_num1'],x['class_num2']=x['class_num2'],x['class_num1']
            x['mag_classn1'],x['mag_classn2']=x['mag_classn2'],x['mag_classn1']
            x['ymmin1'],x['ymmin2']= x['ymmin2'],x['ymmin1']
            x['ymmax1'],x['ymmax2']= x['ymmax2'],x['ymmax1']
            return x
        elif x['ymmax1']<=x['ymmin2']:
            return x
    elif x['move_time']==2:
        if (x['ymmin2']>=x['ymmin1']) & (x['ymmax2']<=x['ymmax1']):
            ym1=x['ym1'][0:-1].split(';')
            temp1 = [t1 for t1 in ym1 if t1<=x['ymmin2']]
            temp2 = [t2 for t2 in ym1 if t2>=x['ymmax2']]
            x['ymmax_new']=x['ymmax1']
            x['ymmax1']=max(temp1)
            x['ymmin_new']=min(temp2)
            return x
        elif  (x['ymmin2']<x['ymmin1']) & (x['ymmax2']>x['ymmax1']):
            ym2=x['ym2'][0:-1].split(';')
            temp1 = [t1 for t1 in ym2 if t1<=x['ymmin1']]
            temp2 = [t2 for t2 in ym2 if t2>=x['ymmax1']]
            x['ymmax_new']=x['ymmax2']
            x['ymmax2']=max(temp1)
            x['ymmin_new']=min(temp2)
            x['city_code1'],x['city_code2']=x['city_code2'],x['city_code1']
            x['ym1'],x['ym2']=x['ym2'],x['ym1']
            x['coauthor1'],x['coauthor2']=x['coauthor2'],x['coauthor1']
            x['class_num1'],x['class_num2']=x['class_num2'],x['class_num1']
            x['mag_classn1'],x['mag_classn2']=x['mag_classn2'],x['mag_classn1']
            x['ymmin1'],x['ymmin2']= x['ymmin2'],x['ymmin1']
            x['ymmax1'],x['ymmax2']= x['ymmax2'],x['ymmax1']
            return x
    elif x['move_time']>2:
        return x        
au_cityn=au_cityn.apply(f, axis=1)
print(au_cityn.shape)

au_cityn2=au_cityn[(au_cityn['ymmin_new'].astype(str)!='0') & (au_cityn['ymmin_new'].astype(str)!='nan')]
def f2(x):
    x['city_code1'],x['city_code2']=x['city_code2'],x['city_code1']
    x['ym1'],x['ym2']=x['ym2'],x['ym1']
    x['coauthor1'],x['coauthor2']=x['coauthor2'],x['coauthor1']
    x['class_num1'],x['class_num2']=x['class_num2'],x['class_num1']
    x['mag_classn1'],x['mag_classn2']=x['mag_classn2'],x['mag_classn1']
    x['ymmin1']=x['ymmin2']
    x['ymmax1']=x['ymmax2']
    x['ymmin2']=x['ymmin_new']
    x['ymmax2']=x['ymmax_new']
    return x
au_cityn2=au_cityn2.apply(f2, axis=1)
print(au_cityn2.shape)

au_cityn= au_cityn.append(au_cityn2)

#计算流动的时间间隔
au_cityn['daysgap']=pd.to_datetime(au_cityn['ymmin2'])-pd.to_datetime(au_cityn['ymmax1'])
au_cityn['daysgap']=au_cityn['daysgap'].map(lambda x:x.days)

au_cityn=au_cityn[['author_first','city_code1','ymmin1','ymmax1','city_code2','ymmin2','ymmax2','move_time','class_overlap','mag_overlap','coauthor_overlap','daysgap']]
print(au_cityn.shape)

au_cityn.to_csv(path+'author_city_to_city_anothermethod.csv',encoding='utf8',index=False)  
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

(357, 15)
   author_first  city_code1  \
0           万广华      5301.0   
1            云鹤      4401.0   
2            付强      3701.0   
3           付文林      3100.0   
4            代谦      3502.0   
5           任寿根      3601.0   
6           何启志      3201.0   
7           何德旭      1100.0   
8           何德旭      1100.0   
9           何德旭      1401.0   
10          何自力      4401.0   
11          何诚颖      4403.0   
12          何雄浪      5101.0   
13           何青      1100.0   
14           余翔      1100.0   
15           俞乔      3100.0   
16          俞红海      3100.0   
17           倪馨      1100.0   
18          冯旭南      3100.0   
19           刘冲      3100.0   

                                                  ym1  \
0   2013-05-01;2008-12-01;2008-04-01;2012-04-01;20...   
1                                         2004-07-01;   
2                                         2017-03-01;   
3                              2014-05-01;2012-05-01;   
4                                         2015-10-01;

(357, 21)
(31, 21)
(388, 12)
0


In [31]:
a=['2010-06-01','2010-06-02','2011-06-01']
x = [k for k in a if k<'2011-06-01' ]
print(x)

['2010-06-01', '2010-06-02']


In [13]:
#步骤9：城市间科研人才流动中的重名现象剔除

#共识别出604968个城市到城市的匹配，如果将a到b，b到a识别成两个（a和b多次交替出现的，没用识别，只标注出move_time），则配对数量为721926
#其中移动一次为43万，移动两次为23万余次,交替3次以上的配对为6万余次

#筛选规则：
#（a）如果两个城市某相同姓名拥有相同的合作者，可以大概率确认这是同一个人，但这个数量较少，12270，占比2%
#     但反之，没有合作者，不代表该姓名就是重名，因为中文社科论文独立作者较多,215万全样本中，63%为独立作者
#（b）如果作者在两个城市间是交替出现，而不是先后出现（保留a到b，b到a的情形），则将其剔除，则大概率是重名问题，
#     或者即便是同一人，但隶属于两个城市的单位，与我们要研究的流动问题不一致。ps：有共同合作者的样本中，城市交替2次及以内占比86.6%
#（c）某姓名下论文中图分类号有重叠，同一人概率较大，有共同合作者的样本中，中图分类号有重叠比例93%，没有共同合作者样本中，重叠比例为36.3%
#（d）某姓名下论文期刊学科领域有重叠，同一人概率较大，有共同合作者的样本中，期刊学科有重叠比例92.9%，没有共同合作者样本中，重叠比例为30.6%
#（e）流动时间间隔越久，重名的可能性越高，有共同合作者的样本中，间隔时间少于5年比例94%，没有共同合作者样本中，该比例为72.8%
#（f）综上，删除交替次数大于2的样本；没有共同合作者，也没有论文中图分类号重叠或期刊学科重叠或间隔时间超过5年
#     换言之，保留样本为交替次数少于等于2次，且有共同合作者，或者没有共同合作者，但论文中图分类号、期刊领域和间隔时间均满足上述要求
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
f1 = open(path+"author_city_to_city_anothermethod.csv",encoding='utf8')
au_cityn= pd.read_csv(f1,header=0,sep=',')
print(au_cityn.shape)
print(au_cityn[0:20])
au_cityn=au_cityn[au_cityn['move_time']<=2]
au_cityn=au_cityn[~(((au_cityn['class_overlap']==0)) & (au_cityn['coauthor_overlap']==0))] 
au_cityn=au_cityn[~(((au_cityn['mag_overlap']==0)) & (au_cityn['coauthor_overlap']==0))] 
au_cityn=au_cityn[~(((au_cityn['daysgap']>365*5)) & (au_cityn['coauthor_overlap']==0))] 
print(au_cityn.shape)  

#A城市-B城市-C城市的流动中，不仅识别了A到B，B到C，还可能识别了A到C，这个理应删除
#删除规则：a到b，a到c两个流动中，保留时间间隔最短的两个
#删除的样本中还包括了一个人对应两个城市的情形，如果将这个情形全部删掉，则删去超过2万个。
au_cityn1=au_cityn[au_cityn['move_time']==1]
au_cityn2=au_cityn[au_cityn['move_time']==2]
au_cityn2['ymmin1']=pd.to_datetime(au_cityn2['ymmin1'])
au_cityn2.sort_values(by=["author_first",'ymmin1','city_code1','daysgap'],inplace=True)
au_cityn2.drop_duplicates(subset=['author_first','ymmin1','city_code1'],keep='first',inplace=True) 
au_cityn= au_cityn1.append(au_cityn2)
print(au_cityn.shape) 

au_cityn.to_csv(path+'author_city_to_city_drop_anothermethod.csv',encoding='utf8',index=False)  
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)


#遗留问题：一个人可能对应两个流向城市；反之亦然，一个人可能来自两个城市

(388, 12)
   author_first  city_code1      ymmin1      ymmax1  city_code2      ymmin2  \
0           万广华      6104.0  2004-08-01  2006-06-01      5301.0  2008-04-01   
1            云鹤      4401.0  2004-07-01  2004-07-01      3301.0  2005-06-01   
2            付强      5000.0  2013-04-01  2013-04-01      3701.0  2017-03-01   
3           付文林      3201.0  2011-07-01  2011-07-01      3100.0  2012-05-01   
4            代谦      4201.0  2006-04-01  2015-05-01      3502.0  2015-10-01   
5           任寿根      3201.0  2001-03-01  2001-03-01      3601.0  2002-01-01   
6           何启志      3201.0  2011-07-01  2011-07-01      3403.0  2017-05-01   
7           何德旭      1100.0  2002-08-01  2006-09-01      1401.0  2010-04-01   
8           何德旭      1100.0  2002-08-01  2006-09-01      4201.0  2009-03-01   
9           何德旭      4201.0  2009-03-01  2009-03-01      1401.0  2010-04-01   
10          何自力      1200.0  2001-02-01  2001-02-01      4401.0  2006-01-01   
11          何诚颖      4403.0  2003-09-01  2

D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\python\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 6.1.3 关系稿

In [14]:
import pandas as pd
import os
import numpy as np
import datetime
starttime = datetime.datetime.now()
#path="D:/python/cnki/" #实际工作
path='D:/python/郭峰Python讲义/数据与结果/cssci/'  #教学演示
fcssci = open(path+"cssci_clean.csv",encoding='utf8')
papers= pd.read_csv(fcssci,header=0,sep=',')
#papers=papers[0:100]
print("剔除单位前数量:"+str(len(papers))) #查看行*列数
papers=papers.dropna(subset=["author_unit"]) #作者单位存在空值，删除
print("剔除空值单位后数量:"+str(len(papers))) #查看行*列数
papers=papers[papers["guanxi_drop"]==1]  #关系稿研究对象是人口迁移的子集
papers=papers[~papers['mag_name'].str.contains('国际汉学')]
print("剔除CSSCI拓展版、隶属关系不清楚或较小的“期刊单位”后的论文数量:"+str(len(papers))) #查看行*列数

#作者单位调整，一些作者单位不规则，比如：中国社会科学院，可能被写成：中国社科院
papers['author_unit_adj']=papers['author_unit']
funitadj= open(path+"unit_name_adj.txt",encoding='utf8')
funitadj=funitadj.read()
for line in funitadj.split('\n'):
    for word in line.split()[1:]:
        if len(word)>0:
            papers["author_unit_adj"] =papers['author_unit_adj'].str.replace(word,line.split()[0])
        else:
            pass
#想看一下单位名称调整的数据多不多,stata结果显示3万个作者单位发生变更
papers['unitadj01']=(papers['author_unit_adj']==papers['author_unit']) 

print("作者单位，调整后：")
print(papers['author_unit_adj'][0:20]) 

#按单位关键词来切割单位，剔除二级单位，只保留一级单位
kus=['大学','政府','检察院','法院','检察院','医院','图书馆','国务院','博物馆','博物院','档案馆','中学','党校','学校','统计局','气象局','电台','电视台','杂志社','报社','集团','公安局','委员会','基金会','协会','研究院','学院','研究所','学会','中心','师专','公司','银行','证券','统计局']
papers['author_unit_short']=papers['author_unit_adj']
papers['author_unit_short']=papers['author_unit_short'].str.replace('(\d+)',"") #去掉邮政编码、数字等。
papers=papers.dropna(subset=["author_unit_short"]) 
for ku in kus: #.split('\n'):
    papers['author_unit_short']=papers['author_unit_short'].str.replace('('+ku+'\S+)',ku)  
print("作者单位，缩写后：")
print(papers['author_unit_short'][0:20])  

papers["author_unit_short"] =papers['author_unit_short'].str.replace("《","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("》","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("（","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("）","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("\"","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("作者系","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("作者单位：","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace(".","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("[","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("]","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("中华人民共和国","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("中国人民解放军","")
papers["author_unit_short"] =papers['author_unit_short'].str.replace("中共","")

#提取第一作者单位，分列:https://www.jianshu.com/p/4a2ecf65e3ea
papers['author_unit_first']=papers['author_unit_short'].str.split(expand=True)[0]
print("作者第一单位，缩写后：")
print(papers['author_unit_first'][0:20])
papers['author_unit_first']=papers['author_unit_first'].str.split(',',expand=True)[0]
papers['author_unit_first']=papers['author_unit_first'].str.split('!',expand=True)[0]
papers['author_unit_first']=papers['author_unit_first'].str.split('、',expand=True)[0]

#定义关系稿#只认第一作者
papers['guanxi1']=(papers['author_unit_first'].str.strip()==papers['mag_unit'].str.strip())  

#另一个关系稿定义，任一作者跟期刊隶属同一单位即可，这里只分析到前10个作者
print("拆分作者单位前列数:"+str(len(papers.columns))) #32列
#begn=len(papers.columns)
papers=pd.merge(papers,pd.DataFrame(papers['author_unit_short'].str.split(expand=True)),how='left',left_index=True,right_index=True)
print("拆分作者单位后列数:"+str(len(papers.columns))) #70列
#endn=len(papers.columns)
papers['guanxi2']=((papers['mag_unit'].str.strip()==papers[0].str.strip())| (papers['mag_unit'].str.strip()==papers[1].str.strip()) |(papers['mag_unit']==papers[2].str.strip())|(papers['mag_unit']==papers[3].str.strip())|(papers['mag_unit']==papers[4].str.strip())|(papers['mag_unit']==papers[5].str.strip())|(papers['mag_unit']==papers[6].str.strip())|(papers['mag_unit']==papers[7].str.strip())|(papers['mag_unit']==papers[8].str.strip())|(papers['mag_unit']==papers[9].str.strip()))

print(papers.shape)
print(papers[0][0:20])
print(papers[1][0:20])
print(papers[2][0:20])
print(papers[3][0:20])
print(papers[4][0:20])

#将第一作者单位与单位城市匹配
funit_city= open(path+"unit_city.csv",encoding='utf8')
unit_citys= pd.read_csv(funit_city,header=0,sep=',')
papers=pd.merge(papers,unit_citys,how='left')
#将匹配成功和未成功样本分隔开
papers1=papers.dropna(subset=["unit_city"]) 
papers2=papers[papers['unit_city'].isnull()]
#将匹配的城市名和代码暂时删除
papers2=papers2.drop(['unit_city','unit_city_code'],axis=1)

#稀有单位的进一步匹配
print("稀有单位匹配前数量:"+str(len(papers2))) #查看行*列数
print(papers2['author_unit_first'][0:20])
funit= open(path+"unit_name.txt",encoding='utf8')
funit=funit.read()
units=funit.split('\n')
for unit in units:
    if len(unit)>=2: #存在一些特殊错误符号，全给替换没了
        papers2['author_unit_first']=papers2['author_unit_first'].str.replace('('+unit+'\S+)',unit) #正则表达式，下列式子存在问题【unit当中不能存在()】

papers2=pd.merge(papers2,unit_citys,how='left')
print(papers2['author_unit_first'][0:20])
print(papers2['unit_city_code'][0:20])
papers= papers1.append(papers2)

#输出部分列
papers_gx=papers[['tlength','mag_name','mag_class','mag_city','mag_city_code','aunum','author_first','author_unit_first','unit_city','unit_city_code','cited','download','fund01','fundn','page_num','year_period','year','month','kwnum','ablength','page_beg','unitadj01','guanxi1','guanxi2']]
papers_gx.to_csv(path+'cssci_gx.csv',encoding='utf8')

endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

剔除单位前数量:8666
剔除空值单位后数量:8390
剔除CSSCI拓展版、隶属关系不清楚或较小的“期刊单位”后的论文数量:8390
作者单位，调整后：
0                                       上海财经大学公共经济与管理学院
1                                          山东财经大学  燕山大学
2                               北京工业大学经济与管理学院  华北水利水电大学
3       北京师范大学经济与工商管理学院  人民大学国家发展与战略研究院  对外经济贸易大学国际经...
4       中国人民大学商学院财务与金融系  武汉大学经济与管理学院博士后流动站  中国华融资产管理...
5             中国科学院大学经济与管理学院  中国科学院大学经济与管理学院中国—丹麦科研教育中心
6                                                上海财经大学
7                               暨南大学产业经济研究院  北京大学光华管理学院
8                                          贵州财经大学工商管理学院
9                                             北京物资学院商学院
10                                            华中科技大学法学院
11                                           重庆大学公共管理学院
12                               安徽财贸职业学院大位会计学院  上海财经大学
13             南开大学经济学院、中国特色社会主义经济建设协同创新中心  清华大学创新发展研究院
14              上海财经大学经济学院  上海财经大学数理经济学重点实验室  美国里海大学经济系
15      中国女企业家协会  中国企业家调查系统  北京卓越企业家成长研究基金会  国务院发展研究...
16                        